In [15]:
import os
import email_reader
import pdfplumber
import email

# Ruta temporal para almacenar archivos adjuntos (dentro de src/)
attachment_dir = "attachments/"
if not os.path.exists(attachment_dir):
    os.makedirs(attachment_dir)

def save_attachments(msg, download_folder=attachment_dir):
    """
    Guarda todos los archivos PDF adjuntos de un correo electrónico.
    """
    pdf_files = []
    
    # Recorremos todas las partes del mensaje de correo
    for part in msg.walk():
        if part.get_content_maintype() == 'multipart':
            continue
        if part.get('Content-Disposition') is None:
            continue
        
        # Obtenemos el nombre del archivo adjunto
        filename = part.get_filename()
        if filename:
            print(f"Archivo adjunto encontrado: {filename}")
            
            if filename.endswith(".pdf"):  # Filtramos solo PDFs
                filepath = os.path.join(download_folder, filename)
                with open(filepath, 'wb') as f:
                    f.write(part.get_payload(decode=True))
                print(f"Archivo PDF guardado: {filepath}")
                pdf_files.append(filepath)
    
    return pdf_files  # Devolvemos la lista de PDFs guardados

# Conectamos a Gmail y seleccionamos todas las carpetas/etiquetas
def fetch_emails_from_all_folders():
    mail = email_reader.connect_to_gmail()
    folders = mail.list()  # Listamos todas las etiquetas disponibles en la cuenta
    if folders:
        for folder in folders[1]:
            # Decodificar el nombre de la carpeta y seleccionarla
            folder_name = folder.decode().split(' "/" ')[1]
            print(f"\nRevisando correos en la carpeta: {folder_name}")
            mail.select(folder_name)

            # Buscar todos los correos en la carpeta seleccionada
            status, messages = mail.search(None, "ALL")

            if status == "OK":
                email_ids = messages[0].split()
                emails = []
                for email_id in email_ids:
                    status, msg_data = mail.fetch(email_id, "(RFC822)")
                    for response_part in msg_data:
                        if isinstance(response_part, tuple):
                            msg = email.message_from_bytes(response_part[1])
                            emails.append(msg)
                return emails
    return []

# Llamamos a la función para revisar todos los correos en todas las carpetas
emails = fetch_emails_from_all_folders()

# Verificamos cuántos correos se encontraron
print(f"Se encontraron {len(emails)} correos en todas las carpetas.")

# Procesar correos y buscar PDFs adjuntos
for idx, email_message in enumerate(emails):
    print(f"\n--- Procesando correo #{idx + 1} ---")
    
    # Guarda todos los archivos PDF adjuntos (si los hay)
    pdf_filepaths = save_attachments(email_message)
    
    # Procesamos todos los archivos PDF encontrados
    for pdf_filepath in pdf_filepaths:
        print(f"Procesando PDF: {pdf_filepath}")
        # Usamos pdfplumber para leer el PDF y extraer el texto
        with pdfplumber.open(pdf_filepath) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                print(text)  # Aquí puedes ajustar para extraer los datos específicos
    
    if not pdf_filepaths:
        print("No se encontró archivo PDF adjunto.")

# Verificamos qué archivos han sido guardados en la carpeta attachments
print("\nArchivos guardados en la carpeta 'attachments':")



Conexión exitosa con Gmail

Revisando correos en la carpeta: "INBOX"
Se encontraron 9 correos en todas las carpetas.

--- Procesando correo #1 ---
No se encontró archivo PDF adjunto.

--- Procesando correo #2 ---
Archivo adjunto encontrado: =?UTF-8?B?VMOpcm1pbm9zIHkgY29uZGljaW9uZXMgd2ViICgxKSAoMSkucGRm?=
Archivo adjunto encontrado: Itinerary (9).pdf
Archivo PDF guardado: attachments/Itinerary (9).pdf
Procesando PDF: attachments/Itinerary (9).pdf
COSTAMAR TRAVEL CRUISE & TOURS BOOKING REF: 2IV2A4
CALLE BERLIN 364 MIRAFLORES DATE: 06 SEPTEMBER 2024
AV PARDO 255
LIMA SAQUICORAY ZARATE/NILTON CESAR MS
PERU
TELEPHONE: 511-616-7777
EMAIL: INFO@CLICKANDBOOK.COM
FLIGHT IB 6652 - IBERIA SUN 20 OCTOBER 2024
OPERATED BY: WAMOS FOR IBERIA
-----------------------------------------------------------------------------
DEPARTURE: LIMA, PE (JORGE CHAVEZ INTL) 20 OCT 11:40
ARRIVAL: MADRID, ES (ADOLFO SUAREZ BARAJAS), TERMINAL 4S - 21 OCT 06:10
4S
FLIGHT BOOKING REF: IB/M77ET
RESERVATION CONFIRMED, ECONO